Obtenemos los datos en formato json desde el API

In [ ]:
!pip install requests

In [ ]:
import requests

r_rios = requests.get(url='https://musicbrainz.org/ws/2/artist/?fmt=json&query=artist:miguel%20rios').json()
r_sabina = requests.get(url='https://musicbrainz.org/ws/2/artist/?fmt=json&query=artist:joaquin%20sabina').json()
r_serrat = requests.get(url='https://musicbrainz.org/ws/2/artist/?fmt=json&query=artist:Joan%20Manuel%20Serrat').json()
r_belen = requests.get(url='https://musicbrainz.org/ws/2/artist/?fmt=json&query=artist:ana%20belen').json()
r_vmanuel = requests.get(url='https://musicbrainz.org/ws/2/artist/?fmt=json&query=artist:victor%20manuel').json()
r_estopa = requests.get(url='https://musicbrainz.org/ws/2/artist/?fmt=json&query=artist:estopa').json()
print(r_rios)
print(r_sabina)
print(r_serrat)
print(r_belen)
print(r_vmanuel)
print(r_estopa)



{'created': '2022-04-28T07:28:45.343Z', 'count': 1522, 'offset': 0, 'artists': [{'id': 'cfd5e4f3-e73d-4f8e-a315-6b99c7b1c3f7', 'type': 'Person', 'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df', 'score': 100, 'gender-id': '36d3d30a-839d-3eda-8cb3-29be4384e4a9', 'name': 'Miguel Ríos', 'sort-name': 'Ríos, Miguel', 'gender': 'male', 'country': 'ES', 'area': {'id': '471c46a7-afc5-31c4-923c-d0444f5053a4', 'type': 'Country', 'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de', 'name': 'Spain', 'sort-name': 'Spain', 'life-span': {'ended': None}}, 'begin-area': {'id': '66ae1ad4-d9df-4213-9447-14eb1298700d', 'type': 'City', 'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78', 'name': 'Granada', 'sort-name': 'Granada', 'life-span': {'ended': None}}, 'disambiguation': 'Spanish singer', 'life-span': {'begin': '1944-06-07', 'ended': None}, 'aliases': [{'sort-name': 'Rios, Miguel', 'name': 'Miguel Rios', 'locale': None, 'type': None, 'primary': None, 'begin-date': None, 'end-date': None}, {'sort-name':

In [ ]:
list_artists = [r_rios, r_sabina, r_serrat, r_belen, r_vmanuel, r_estopa]
data_detail = {}
doc_final = {}
doc_final['artists'] = []
list_ids = []
for artist in list_artists:
  artist_add = {}
  artist_add['id'] = artist['artists'][0]['id']
  artist_add['name'] = artist['artists'][0]['name'];
  artist_add['country'] = artist['artists'][0]['area']['name'];
  if artist['artists'][0]['type'] == 'Person':
    artist_add['gender'] = artist['artists'][0]['gender'];
    artist_add['type'] = 'author';
  else:
    artist_add['gender'] = 'NA'
    artist_add['type'] = artist['artists'][0]['type'];
  artist_add['city'] = artist['artists'][0]['begin-area']['name'];
  doc_final['artists'].append(artist_add);
  data_detail[artist['artists'][0]['id']] =  requests.get(url='https://musicbrainz.org/ws/2/release?fmt=json&artist=' + 
                                                          artist['artists'][0]['id'] + '&limit=100&inc=artist-credits+recordings').json()
  list_ids.append(artist['artists'][0]['id']) 
 


print(doc_final)
print(data_detail)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
list_cols = []
list_records = []
for i,artist in enumerate(list_artists):
  colaborations = []
  colaborations_albums = []
  list_cols.append(colaborations)
  data = data_detail[doc_final['artists'][i]['id']]
  id_data = doc_final['artists'][i]['id']
  recording_cols = {}
  recordings = {}
  list_records.append(recordings)
  for j,id in enumerate(list_ids):
    if(id!=id_data):
      recordings[id_data] = []
      recording_cols[id_data+'+'+id] = {}
      ms = [r for r in data['releases']]
      for item in ms:
        recordings[id_data].append(item['title'])
        for media in item['media']:
          for track in media['tracks']:
            for rec in track['recording']:
              for col in track['artist-credit']:
                if(col['artist']['id'] == id):
                  if item['title'] not in recording_cols[id_data+'+'+id]:
                    recording_cols[id_data+'+'+id][item['title']] = 1  
                  else:
                    recording_cols[id_data+'+'+id][item['title']]+1
                     
      colaborations.append(len(recording_cols[id_data+'+'+id]))
    
    else:
      colaborations.append(0)


list_cols_fin = [[0,0,0,0,0,0],[0,0,0,0,0,0],
                 [0,0,0,0,0,0],[0,0,0,0,0,0],
                 [0,0,0,0,0,0],[0,0,0,0,0,0]]
for i in range(0,6):
   for j in range(0,6):
     if(i!=j):
       list_cols_fin[i][j] = list_cols[j][i] + list_cols[i][j]

print(list_cols_fin)
for i,artist in enumerate(doc_final['artists']):
      artist['colaborations'] = list_cols_fin[i]
      artist['record_list'] = list_records[i][artist['id']]
      artist['number_releases'] = len(list_records[i][artist['id']])

print(doc_final)

[[0, 1, 1, 4, 3, 0], [1, 0, 0, 1, 1, 2], [1, 0, 0, 2, 1, 1], [4, 1, 2, 0, 15, 1], [3, 1, 1, 15, 0, 0], [0, 2, 1, 1, 0, 0]]
{'artists': [{'id': 'cfd5e4f3-e73d-4f8e-a315-6b99c7b1c3f7', 'name': 'Miguel Ríos', 'country': 'Spain', 'gender': 'male', 'type': 'author', 'city': 'Granada', 'colaborations': [0, 1, 1, 4, 3, 0], 'record_list': ['Al Andalus', 'El rock de una noche de verano', 'Directo al corazón', 'Como el viento / Ella se fue', 'Solo o en compañía de otros', 'Miguel Ríos y las estrellas del rock latino', 'Himno a la alegría', 'A Song of Joy / El Rio', 'Grandes éxitos', '60 Mp3', 'Rock & Ríos', 'Extraños en el escaparate', 'Despierta', 'Vuelvo a Granada', 'Hino da alegria', 'La encrucijada', 'Miguel Ríos', 'El rock de una noche de verano', '¡Qué noche la de aquel año!', 'El río / Vuelvo a Granada', 'Rock & Ríos', 'Éxitos originales', 'Big Band Ríos', 'Rock & Ríos', 'Madre Tierra', 'Así que pasen 30 años', 'De colección', 'Rocanrol Bumerang', 'Rocanrol Bumerang', 'Los viejos rockeros

In [ ]:
import json

print(doc_final)
with open("network.json", "w") as outfile:
    json.dump(doc_final, outfile)

{'artists': [{'id': 'cfd5e4f3-e73d-4f8e-a315-6b99c7b1c3f7', 'name': 'Miguel Ríos', 'country': 'Spain', 'gender': 'male', 'type': 'author', 'city': 'Granada', 'colaborations': [0, 1, 1, 4, 3, 0], 'record_list': ['Al Andalus', 'El rock de una noche de verano', 'Directo al corazón', 'Como el viento / Ella se fue', 'Solo o en compañía de otros', 'Miguel Ríos y las estrellas del rock latino', 'Himno a la alegría', 'A Song of Joy / El Rio', 'Grandes éxitos', '60 Mp3', 'Rock & Ríos', 'Extraños en el escaparate', 'Despierta', 'Vuelvo a Granada', 'Hino da alegria', 'La encrucijada', 'Miguel Ríos', 'El rock de una noche de verano', '¡Qué noche la de aquel año!', 'El río / Vuelvo a Granada', 'Rock & Ríos', 'Éxitos originales', 'Big Band Ríos', 'Rock & Ríos', 'Madre Tierra', 'Así que pasen 30 años', 'De colección', 'Rocanrol Bumerang', 'Rocanrol Bumerang', 'Los viejos rockeros nunca mueren', 'La Huerta Atómica (Un Relato de Anticipación)', 'Cantan a Kurt Weill', 'Como si fuera la primera vez', 'Ca